## Исследование производительности работы с хранилищем MongoDB

In [2]:
import uuid
import time

from typing import Any
from functools import wraps
from random import randint, choice
from motor.motor_asyncio import AsyncIOMotorClient

In [3]:
client = AsyncIOMotorClient('mongodb://localhost:27017')
database = client["moviesDb"]
collection = database["movies"]

await collection.create_index([('event_name', 1),])
await collection.create_index([('event_name', 1), ('movie_id', 1), ])
await collection.create_index([('event_name', 1), ('user_id', 1), ])

'event_name_1_user_id_1'

In [4]:
def benchmark(operation: str):
    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            start_time = time.perf_counter()
            func_result = await func(*args, **kwargs)
            end_time = time.perf_counter()
            avg_time = end_time - start_time
            print(f"Время выполнения функции {operation} заняло {avg_time} cек.")
            return func_result
        return wrapper
    return decorator

In [27]:
def generate_like():
    return {
        'event_name': 'like',
        'user_id': str(uuid.uuid4()),
        'movie_id': str(uuid.uuid4()),
        'score': randint(0, 10)
    }


def generate_bookmark():
    return {
        'event_name': 'bookmark',
        'user_id': str(uuid.uuid4()),
        'movie_id': str(uuid.uuid4()),
        'score': None
    }


async def generate_data_in_mongo(
    generator: Any,
    docs_count: int,
    batch_size: int = 100
):
    docs = []
    for _ in range(docs_count):
        docs.append(generator())
        if len(docs) >= batch_size:
            await collection.insert_many(docs)
            docs = []
    if docs:
        await collection.insert_many(docs)


@benchmark(operation='Выбор рандомного списка элементов')
async def get_random_event_docs(
    event_name: str,
    docs_count: int = 10
) -> list[dict]:
    cursor = collection.aggregate(
		[
			{'$match': {'event_name': event_name}},
            {'$sample': {'size': docs_count}},
        ]
    )
    return [_ async for _ in cursor]

In [28]:
@benchmark(operation='Получение количества лайков для конкретного фильма')
async def get_count_of_likes_for_movie(
    movie_id: uuid.UUID
) -> None:
    return await collection.count_documents(
        {'event_name': 'like', 'movie_id': str(movie_id)}
    )


@benchmark(operation='Получение списка закладок для рандомного пользователя')
async def get_user_bookmarks(
    user_id: uuid.UUID,
) -> None:
    await collection.aggregate(
        [
			{
                '$match': {
                    'event_name': 'bookmark',
                    'user_id': str(user_id)
                }
            },
			{'$limit': 10}
        ]
    ).to_list(10)


@benchmark(operation='Добавление лайка пользователя для фильма')
async def add_like_to_movie(
    user_id: uuid.UUID,
    movie_id: uuid.UUID
) -> None:
    await collection.insert_one(
		{
            'event_name': 'like',
            'user_id': str(user_id),
            'movie_id': str(movie_id)
        }
    )


@benchmark(operation='Удаление лайка пользователя для фильма')
async def delete_like_from_movie(
    user_id: uuid.UUID,
    movie_id: uuid.UUID
) -> None:
    await collection.delete_one(
		{
            'event_name': 'like',
            'user_id': str(user_id),
            'movie_id': str(movie_id)
        }
    )

In [31]:
for _ in range(100):
    await generate_data_in_mongo(generate_like, 25_000, 25_000)
    await generate_data_in_mongo(generate_bookmark, 25_000, 25_000)

random_like_docs = await get_random_event_docs('like', docs_count=2)
random_bookmark_docs = await get_random_event_docs('bookmark', docs_count=2)

await get_count_of_likes_for_movie(choice(random_like_docs)['movie_id'])
await get_user_bookmarks(choice(random_bookmark_docs)['user_id'])

random_user_id, random_movie_id = (uuid.uuid4(), uuid.uuid4())
await add_like_to_movie(random_user_id, random_movie_id)
await delete_like_from_movie(
    choice(random_like_docs)['user_id'],
    choice(random_bookmark_docs)['movie_id']
)

Время выполнения функции Выбор рандомного списка элементов заняло 11.03542974299853 cек.
Время выполнения функции Выбор рандомного списка элементов заняло 10.494517550998353 cек.
Время выполнения функции Получение количества лайков для конкретного фильма заняло 0.0019902130006812513 cек.
Время выполнения функции Получение списка закладок для рандомного пользователя заняло 0.0011761360001401044 cек.
Время выполнения функции Добавление лайка пользователя для фильма заняло 0.0011731509985111188 cек.
Время выполнения функции Удаление лайка пользователя для фильма заняло 0.0015051609989313874 cек.


### Вывод
Скорость чтения данных из MongoDB укладывается в требования (200мс)